In [48]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [49]:
STATIONS = ['S32', 'S33']
train_date_part = pd.read_csv('input_orig/train_date.csv', nrows=10000)

In [50]:
date_cols = train_date_part.drop('Id', axis=1).count().reset_index().sort_values(by=0, ascending=False)

In [51]:
date_cols['station'] = date_cols['index'].apply(lambda s: s.split('_')[1])

In [52]:
date_cols = date_cols[date_cols['station'].isin(STATIONS)]

In [53]:
date_cols = date_cols.drop_duplicates('station', keep='first')['index'].tolist()
print(date_cols)

['L3_S33_D3870', 'L3_S32_D3852']


In [54]:
train_date = pd.read_csv('input_orig/train_date.csv', usecols=['Id'] + date_cols)

In [55]:
train_date.columns = ['Id'] + STATIONS

In [56]:
print(train_date.columns)

Index(['Id', 'S32', 'S33'], dtype='object')


In [57]:
for station in STATIONS:
    train_date[station] = 1 * (train_date[station] >= 0)

In [58]:
response = pd.read_csv('input_orig/train_numeric.csv', usecols=['Id', 'Response'])
print(response.shape)

(1183747, 2)


In [59]:
train = response.merge(train_date, how='left', on='Id')
# print(train.count())
train.head()

,Id,Response,S32,S33
0,4,0,0,1
1,6,0,0,1
2,7,0,0,1
3,9,0,0,1
4,11,0,0,1


In [60]:
train['cnt'] = 1

In [61]:
train.head()

,Id,Response,S32,S33,cnt
0,4,0,0,1,1
1,6,0,0,1,1
2,7,0,0,1,1
3,9,0,0,1,1
4,11,0,0,1,1


In [62]:
failure_rate = train.groupby(STATIONS).sum()[['Response', 'cnt']]

In [63]:
failure_rate['failure_rate'] = failure_rate['Response'] / failure_rate['cnt']

In [64]:
failure_rate

Response      cnt  failure_rate
S32 S33                                 
0   0         319    62485      0.005105
    1        5454  1096719      0.004973
1   0        1014     6567      0.154408
    1          92    17976      0.005118

In [ ]:
failure_rate = failure_rate[failure_rate['cnt'] > 1000]  # remove 
failure_rate.head(20)